In [193]:
import pickle as pkl
import numpy as np
from os import walk
from sklearn.model_selection import train_test_split
import os

In [194]:
def readFileNames(Readpath:str) -> list:
    files = []
    for (dirpath, dirnames, filenames) in walk(Readpath):
        files.extend(filenames)
    print(f'read {len(files)} files ...')
    return files

In [195]:
moyoDataDir = './mosh/train/'
files = readFileNames(moyoDataDir)
moyotrain,moyotest = train_test_split(files,test_size=0.3,shuffle=True)

read 209 files ...


In [196]:
def delete_files_in_folder(folder_path):    
    # 遍历文件夹中的所有文件并删除
    print(f'Start to delete files in {folder_path}')
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        try:
            if os.path.isfile(file_path):
                os.remove(file_path)
                #print(f"deleted file: {file_path}")
        except Exception as e:
            print(f"error when delete'{file_path}' : {e}")
    print(f'Delete {len(filename)} files succesfully!')

In [197]:
def convertfiles(readPath: str, savePath: str, files: list): 
    if not os.path.exists(savePath):
        os.makedirs(savePath)
    else:
        delete_files_in_folder(savePath)
    
    print(f'start load {len(files)} from {readPath} to {savePath} ...')
    for filename in files:
        with open(readPath+filename,'rb') as f:
            #print("load data from {}".format(f.name))
            data = pkl.load(f)
            joint_pos = data['body_pose']
            global_orient = data['global_orient']
            poses = np.concatenate((joint_pos,global_orient), axis = 1)
            poses = np.array([poses])
            with open(savePath+filename,'wb') as fo:
                fo = pkl.dump({'poses':poses}, fo)
    print(f'load files ok!')

In [198]:

def splitMoyoData(readDir: str, saveDir: str, )->None:
    print('Preparing Moyo Data...')
    # train folder
    readPath = readDir+ 'train/'
    files = readFileNames(readPath)
    moyotrain,moyotest = train_test_split(files,test_size=0.3,shuffle=True)
    print('===============train===============')
    savePath = saveDir+ 'train/'
    convertfiles(readPath,savePath,moyotrain)
    print('===============test================')
    savePath = saveDir+ 'test/'
    convertfiles(readPath,savePath,moyotest)
    # val folder
    print('===================================')
    readPath = readDir+ 'val/'
    files = readFileNames(readPath)
    print('================val================')
    savePath = saveDir+ 'validation/'
    convertfiles(readPath,savePath,files)
    print(f'Moyo Data prepared!')


In [199]:
readPath = './dataset/mosh_smpl/'
savePath = './dataset/mosh/'
splitMoyoData(readPath,savePath)

Preparing Moyo Data...
read 299 files ...
===============train===============
Start to delet files in ./mosh/train/
Delete 108 files succesfully!
start load 209 from ./mosh_smpl/train/ to ./mosh/train/ ...
load files ok!
===============test================
Start to delet files in ./mosh/test/
Delete 97 files succesfully!
start load 90 from ./mosh_smpl/train/ to ./mosh/test/ ...
load files ok!
read 36 files ...
================val================
Start to delet files in ./mosh/validation/
Delete 77 files succesfully!
start load 36 from ./mosh_smpl/val/ to ./mosh/validation/ ...
load files ok!
Moyo Data prepared!


In [ ]:
!rm -rf ./dataset/mixed/
!cp -r ./dataset/mosh/ ./dataset/mixed
!cp -r ./dataset/3DPW/sequenceFiles/* ./dataset/mixed

In [ ]:
!python main_3dpw.py --data_dir_3dpw "./dataset/mixed/" --input_n 10 --output_n 30 --dct_n 40 --exp "./logs/"